In [38]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=2)

## MLP engine

In [39]:
class MLP():
    def __init__(self, listOfLayer, activationFunction = "ReLU"):
        self.numLayers = len(listOfLayer)
        self.layers = [{} for i in range(self.numLayers)]
        self.activationFunction = activationFunction
        
        for i in range(1, self.numLayers):
            self.layers[i]["W"] = np.random.randn(listOfLayer[i], listOfLayer[i-1])  # weight (random)
            self.layers[i]["b"] = np.random.randn(listOfLayer[i], 1)  # bias (random)
            
        self.lossList = []  # loss value trace
        return
    
    def activation(self, z, actType="sigmoid"):
        if actType == "sigmoid":
            a = 1.0 / (1.0 + np.exp(-z))
        elif actType == "ReLU":
            a = np.where(z < 0.0, 0.0, z)  
            
        return a
    
    def inverseActivation(self, a, actType="sigmoid"):
        if actType == "sigmoid":
            dzda = a * (1.0 - a)
        elif actType == "ReLU":
            dzda = np.where(a > 0, 1.0, a)
            
        return dzda
    
    def forward(self, x, bs):
        self.layers[0]["a"] = np.copy(x)
        
        #Linear equation
        for i in range(1, self.numLayers):
            self.layers[i]["z"] = self.layers[i]["W"].dot(self.layers[i-1]["a"]) + self.layers[i]["b"]
            actType = "sigmoid" if i == self.numLayers - 1 else self.activationFunction
            self.layers[i]["a"] = self.activation(self.layers[i]["z"], actType)
            
        # last layer L does not use activation function    
        self.p = self.softmax(self.layers[-1]["a"])
        self.yHat = np.zeros(self.p.shape, dtype=int)  # predicted answers
        for j, i in enumerate(self.p.argmax(axis=0)):
            self.yHat[i, j] = 1
        return
    
    def softmax(self, a):
        expa = np.exp(a)
        return (expa / np.sum(expa, axis=0))
    
    def loss(self, y):
        return np.mean(- np.log(self.p) * y)
    
    def backprop(self, y, bs):
        # Derivative of loss function
        self.dJdp = - y / self.p
        
        # Derivative of softmax
        dpda = np.zeros((self.p.shape[0], self.p.shape[0], bs)) # a and p are of same dimension
        for b in range(bs):
            for i in range(dpda.shape[0]):
                for j in range(dpda.shape[1]):
                    dpda[i, j, b] = (self.p[i, b] - self.p[i, b] ** 2) if i == j else - self.p[i, b] * self.p[j, b]
        
        self.layers[-1]["dJda"] = np.zeros(self.layers[-1]["a"].shape)
        for b in range(bs):
            self.layers[-1]["dJda"][:, b] = dpda[:, :, b].dot(self.dJdp[:, b])
          
        for i in range(self.numLayers - 1, 0, -1):
            actType = "sigmoid" if i == self.numLayers - 1 else self.activationFunction
            self.layers[i]["dJdz"] = self.inverseActivation(self.layers[i]["a"], actType) * self.layers[i]["dJda"]
            self.layers[i]["dJdb"] = np.mean(self.layers[i]["dJdz"], axis = 1).reshape(self.layers[i]["b"].shape)
            self.layers[i]["dJdW"] = self.layers[i]["dJdz"].dot(self.layers[i-1]["a"].T) / bs
            self.layers[i-1]["dJda"] = self.layers[i]["W"].T.dot(self.layers[i]["dJdz"])
        return
    
    def update(self, lr):
        # lr: learning rate
        for i in range(1, self.numLayers):
            self.layers[i]["W"] -= lr * self.layers[i]["dJdW"]
            self.layers[i]["b"] -= lr * self.layers[i]["dJdb"]
        return
    
    def shuffle(self, a, b):
        shuffled_a = np.copy(a)
        shuffled_b = np.copy(b)
        permutation = np.random.permutation(a.shape[1])
        for oldindex, newindex in enumerate(permutation):
            shuffled_a[:, oldindex] = a[:, newindex]
            shuffled_b[:, oldindex] = b[:, newindex]
        return shuffled_a, shuffled_b
        
    def train(self, trainX, trainY, numEpoch=1, lr=0.01, bs=2):  
        for e in range(numEpoch):
            shuffled_trainX, shuffled_trainY = self.shuffle(trainX, trainY)
            for i in range(trainX.shape[1] // bs):
                x = shuffled_trainX[:, i*bs : (i+1)*bs]
                y = shuffled_trainY[:, i*bs : (i+1)*bs]
                self.forward(x, bs)
                self.lossList.append(self.loss(y))
                self.backprop(y, bs)
                self.update(lr)
                
        self.finalX = shuffled_trainX
        self.finalY = shuffled_trainY
        return      

## Forward
Linear equation: $$z^l = w^l a^{l-1} + b^l, \forall l \neq 0$$
ReLU activation function: $$a^l_i = \max(z^l_i, 0), \forall i, 0<l<output\_layer$$
Softmax function: $$p_i = \frac{e^{z^L_i}}{\Sigma_j e^{z^L_j}}, \forall i$$

## Backward
Corss entropy loss function: $$J = -\sum_i (y_i \ln p_i + (1-y_i) \ln (1-p_i))$$
Derivative of cross entropy loss function: $$\frac{\partial J}{\partial p} = (\frac{1}{1-y-p})^\text{T} = dJdp$$
Derivative of softmax: $$\frac{\partial p}{\partial z^L} = \text{diag}(p)-pp^\text{T} = dpdz$$
Derivative of ReLU activation function: $$\text{diag}^{-1}(\frac{\partial a^l}{\partial z^l}) = z^l > 0 ? 1:0 = diag\_inv\_dadz[l]$$
Recursive formula by chain rule: 
$$\frac{\partial J}{\partial z^L} = \frac{\partial J}{\partial p} \frac{\partial p}{\partial z^L} \\
\Rightarrow dJdz[last\_layer] = dJdp \times dpdz$$
$$\frac{\partial J}{\partial z^l} = (\frac{\partial J}{\partial z^{l+1}} w^{l+1}) \otimes {\text{diag}^{-1}(\frac{\partial a^l}{\partial z^l})}^\text{T} \\
\Rightarrow dJdz[l] = (dJdz[l+1] \times w[l+1]) \otimes diag\_inv\_dadz[l]^\text{T}$$
Finally... 
$$\frac{\partial J}{\partial w^l} = a^{l-1} \frac{\partial J}{\partial z^l} = dJdw[l]$$
$$\frac{\partial J}{\partial b^l} = \frac{\partial J}{\partial z^l} = dJdb[l]$$

## Training block
### train 60k dataset

In [40]:
f = open("mnist_train_60000.csv", "r")
train_60k = f.readlines()
f.close()

x = [] 
y = []  # answers

for line in train_60k:
    linepixels = [int(pixel) for pixel in line.split(",")]
    x.append(linepixels[1:])
    y.append(linepixels[0])
    
train_x_60k = np.array(x).T / 256.0
train_y_60k = np.zeros((10, len(y)), dtype=int)
for i in range(len(y)):
    train_y_60k[y[i], i] = 1

In [45]:
nnMNIST60k = MLP([784, 80, 10], activationFunction = "sigmoid")
nnMNIST60k.train(train_x_60k, train_y_60k, numEpoch=1000, lr=0.2, bs=500)

In [46]:
nnMNIST60k.forward(train_x_60k, 60000)
predicted = nnMNIST60k.yHat.argmax(axis=0)

## Training Accuracy

In [47]:
y_ = np.array(y)
print("Training Accuracy=", np.sum(y_ == predicted) / y_.shape[0])

Training Accuracy= 0.9419333333333333


## Testing block
### test 10k dataset

In [48]:
f2 = open("mnist_test_10000.csv", "r")
test_10k = f2.readlines()
f2.close()

x_test = [] 
y_test = []  # answers

for line in test_10k:
    linepixels = [int(pixel) for pixel in line.split(",")]
    x_test.append(linepixels[1:])
    y_test.append(linepixels[0])

test_x_10k = np.array(x_test).T / 256.0

## Testing Accuracy

In [50]:
nnMNIST60k.forward(test_x_10k, 10000)
predicted_t = nnMNIST60k.yHat.argmax(axis=0)

y_t = np.array(y_test)
print("Testing Accuracy=", np.sum(y_t == predicted_t) / y_t.shape[0])

Testing Accuracy= 0.9375
